# Overview 

Today's class will have two parts: 

First, we will review the homework, and describe ways to limit the amount of memory used in loading large data sets. 

Second, we will describe the structure of the experiment that produced the data we have been analyzing, and we will compute averages of activity around the time of specific experimental events. 


# Goals
* Understand ways to reduce the amount of memory used when loading data
* Understand *masking* data with logical indices
* Estimate the average response to an experimental event

# Updating resources in your server home directory
(Run the cells in this section once, then restart your kernel, reload the web page, and skip this section the next time through!)

In [ ]:
# Updating functions
import neurods
# Update neurods package
neurods.io.update_neurods()

# Memory management and masking
A big difficulty in last week's homework - and in data science in general - is how to deal with large data sets. 

In [ ]:
# Load some necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import nibabel
import neurods
import cortex
import os

In [ ]:
# Set plotting defaults
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
# Set matplotlib defaults!
plt.rcParams['image.cmap'] = 'viridis'
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.origin'] = 'lower'
plt.rcParams['image.aspect'] = 'equal'

### Python digression: Floating point vs integer numbers

`numpy` stores numbers in several different formats: numbers can be stored as boolean values (True or False); as integers (0, 1, 2...) or as floating-point numbers (2.3256..., 3.63212..., etc). This is a common aspect of all programming languages that deal with images or numbers. Different formats for numbers use different amounts of memory. For data types that allow decimals (e.g. numpy's float32 and float64), the more decimal places that are stored for each number in an array, the more memory the array takes up. 

Thus, converting to a less-precise format (np.float32) can save memory, if precision is not critically important.

In [ ]:
print(np.float64(np.pi))
print(np.float32(np.pi))

In [ ]:
r64 = np.random.rand(30,100,100)
r32 = r64.astype(np.float32)
print('data type of `r64` is: ', r64.dtype)
print('data type of `r32` is: ', r32.dtype)

In [ ]:
whos

### HW Recap

In [ ]:
from scipy.stats import zscore

# An OK implementation of load_data:
from scipy.stats import zscore
def load_data_ok(files, do_zscore=False):
    """Load fMRI data from files and optionally z-normalize data"""
    # Create a list to store data
    data = None
    for f in files:
        nii = nibabel.load(f)
        if data is None:
            data = nii.get_data().T
            if do_zscore:
                data = zscore(data, axis=0)
        else:
            tmp = nii.get_data().T
            if do_zscore:
                tmp = zscore(tmp, axis=0)            
            data = np.vstack([data, tmp])
    return data

# A better implementation
def load_data_better(files, do_zscore=False):
    """Load fMRI data from files and optionally z-normalize data
    
    Parameters
    ----------
    files : list 
        List of file names (absolute paths)
    do_zscore : bool
        Flag that determines whether to zscore data in time or not
    
    Returns
    -------
    data : array
        fMRI data array, in (time, z, y, x) format
    """
    # Create a list to store data
    data = []
    # Loop over files in list
    for f in files:
        nii = nibabel.load(f)
        tmp = nii.get_data().T
        # Optionally zscore each run independently
        if do_zscore:
            data = zscore(data, axis=0)
        data.append(tmp)
    # Concatenate full data
    data = np.vstack(data)
    return data

In [ ]:
### STUDENT ANSWER


In [ ]:
### STUDENT ANSWER


In [ ]:
### STUDENT ANSWER


In [ ]:
ls /Users/mark/Dropbox/data8/fMRI/motor/

In [ ]:
### STUDENT ANSWER


# MOVE ME


Consequently, it is common to display the results of statistical analyses of fMRI data on inflated and flattened representations of the cerebral cortex. Such cortical surface maps provide a way to examine all cortical fMRI data at once, with the anatomical location of the functional data made clear. 

The cortical surface must be computationally extracted from high spatial resolution anatomical MRI scans, and often manually edited (*NOTE: Manual editing to create a good corical surface can take days or weeks of effort! This data is not free!*)

<img src="figures/MPRAGE.png" align='left' style="height: 200px;">

<img src="figures/MPRAGE_wcortex.png" align='left' style="height: 200px;">

<img src="figures/cortex_3views.png" align='left' style="height: 200px;">

## Masking

As we have discussed, not all of the data in our 4D array is equally interesting to us. We are interested in the fMRI data collected IN the brain (vs outside it), and more specifically in the data collected from the cerebral cortex (the outermost layer of the brain). 

Here, we will show you how to extract (a) the data in the brain, and (b) the data in the cerebral cortex from the whole array. 

Remember our histogram of values for data, which show a ton of voxels with zero values (from outside the brain):

In [ ]:
# Specify files
files = ['s01_categories_{:02d}.nii.gz'.format(r) for r in [1, 2, 3]]
files = [os.path.join(neurods.io.data_list['fmri'], 'categories', f) for f in files]

In [ ]:
# Load data for only one file
data = load_data(files[0], do_zscore=False)

In [ ]:
bins = np.linspace(0,2000,31)
_ = plt.hist(data.flatten(), bins)
plt.xlabel('Raw fMRI Activity')
plt.ylabel('TRs (count)')

So: how can we extract the data that is only from the region of the scan that contains the brain? We could try to write down an index for each data point in the data that contains a brain voxel (e.g. [25, 33, 33], [25, 33, 34]), but you can see how such a list would get quite long (tens of thousands) and would be difficult to construct. 

One simple way to find data that is in or near the brain is to threshold the data to find only the voxels where the signal is greater than zero. 

In [ ]:
# Here, consider only the first volume
brain_voxels = data[0] > 0
print(brain_voxels)

In [ ]:
### STUDENT ANSWER


In [ ]:
# What is this thing we have just created?
print('dtype of `brain_voxels`: ', brain_voxels.dtype)
print('Sum of of `brain_voxels`: ', brain_voxels.sum())
print('Mean of of `brain_voxels`: ', brain_voxels.mean())
print('Shape of `brain_voxels`: ', brain_voxels.shape)

### Breakout session
1. Discuss what each of the values above indicate about the `brain_voxels` array.
2. What happens if you change the cell above to be brain_voxels = data[0] > X, where X is greater than zero? (What should the threshold [X] for selecting brain voxels be?)
3. While playing with the threshold value, display the `brain_voxels` variable in some sensible way. What does the array LOOK like for different thresholds (values of X)?

In [ ]:
### STUDENT ANSWER


Now we have an array of True/False values (a boolean array). This array can be directly used to INDEX our data! 

In [ ]:
# Logical indices are fun!
a = np.arange(10)
idx = np.array([True, False, True, False, True, False, True, False, True, False])
a[idx]

In [ ]:
# This works in multiple dimensions, too!
a = np.arange(20).reshape(2,10)
print(a)

In [ ]:
print(a[:,idx])

In [ ]:
# or even for brain data!
brain_data = data[:, brain_voxels]
print(brain_data.shape)

### BREAKOUT SESSION
Make a histogram of `brain_data`. Z-score it, and plot it as an image.

In [ ]:
### Student answer
plt.hist(brain_data.flatten(), bins)
plt.xlabel('Raw BOLD response')
plt.ylabel('TRs (count)')
plt.figure()
plt.imshow(zscore(brain_data, axis=0), aspect='auto')
plt.xlabel("Voxels")
plt.ylabel("Time (TRs)")

In [ ]:
# Undo masking action?

# Masking with pycortex

In [ ]:
# Just like specifying a volume, pycortex needs a subject and a transform to retrieve a mask
# for a particular data set.
cortical_voxels = cortex.db.get_mask('s01', 'catloc', type='cortical')

In [ ]:
# Display the same information as the brain mask above
print(cortical_voxels.dtype)
print(cortical_voxels.shape)
print(cortical_voxels.sum())
print(cortical_voxels.mean())

In [ ]:
# Plot 
fig1 = plt.figure(figsize=(6,5))
_ = neurods.viz.slice_3d_array(cortical_voxels, axis=0, fig=fig1)

In [ ]:
### TEACHER INFO
fig2 = plt.figure(figsize=(10,3))
_ = neurods.viz.slice_3d_array(cortical_voxels, axis=1, fig=fig2)

In [ ]:
cortical_data = data[:, cortical_voxels]
print(cortical_data.shape)

In [ ]:
whos

## Load description of experiment 
The experiment we have been working with is a *localizer* experiment. It is designed to find areas of the brain that respond to particular visual categories of objects: faces, bodies, and places. It also reveals areas that respond more to objects than to scrambled versions of the same objects. This experiment is a simple replication of past work, and is commonly done as a first step to locate (or localize) a region of interest for further analysis in a subsequent experiment.

For the localizer experiment, images from each category were presented in a block design. This means that images from the one category were shown one after another for a "block" of 20 seconds (10 TRs), followed by images from another category for a block of 20 seconds, and so on.

<img src="figures/CategoryLocalizerDesign.001.png" style="height: 400px;">

To analyze the data from this experiment at all, we need to know when the blocks for each category (faces, bodies, places, objects, and scrambled objects) began and ended. This information is stored in a *design matrix*, which we load below.

In [ ]:
import numpy as np
import os
import neurods

In [ ]:
basedir = os.path.join(neurods.io.data_list['fmri'], 'categories')
design = np.load(os.path.join(basedir, 'experiment_design.npz'))
print('Experiment design variables: ', sorted(design.keys()))

In [ ]:
conditions = design['conditions'].tolist()
print('Conditions: ', conditions)
design_run1 = design['run1']
print('Design shape: ', design_run1.shape)

It's often useful to show a design matrix as an image:


In [ ]:
_ = plt.imshow(design_run1.T, aspect='auto')

## Breakout session
> What are the dimensions here? Label the axes on the figure above!

# Event-related averages
In an experiment, we are always interested in the relationship between the stimulus and brain responses. The simplest way to visualize the relationship is to examine what happened to brain responses every time that a stimulus came on. Thus, we will now create *averages* of responses after a particular type of stimulus came on.

In [ ]:
# First, use np.nonzero to find condition onsets for the first condition
_, on_times = np.nonzero([design_run1[:,0]])
print(on_times)

## Breakout session
> Convert `on_times` to onsets! (i.e., the specific time that the stimulus came on)

> Select 10 time points after each time the stimulus came on, and save them in a list

> Average each set of points together! 

In [ ]:
### STUDENT ANSWER


In [ ]:
# Make a movie of the temporal average in pycortex!
sub, xfm = 's01', 'catloc'
cond1_vol = cortex.Volume(data_avg_cond1, sub, xfm, vmin=-3, vmax=3, cmap='RdBu_r')
cx.webgl.show(cond1_vol)